In [ ]:
# The motivation for this notebook is to investigate the correlation between Covid-19 cases and deaths in US counties and specific social vulnerability indicators. 
# The latters are part of the Social Vulnerability Index (SVI) that US agency CDC provides every two years. It includes indicators like poverty, minority, single-parent, etc. 
# on a county level. 
# For my research I have selected 14 indicators that all show percentage values, plus an overall ranking variable called RPL_THEMES. 

# After investigating the correlation with Covid-19 cases and deaths, it is my plan to build a simple Regression model that is based on social vulnerability indicators 
# to learn more about their correlation. 



# Importing libraries

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import datetime
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
# import imgkit #necessary to export Pandas Style objects to imgs. 

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))


# Importing data

svi = pd.read_csv('/kaggle/input/cdc-social-vulnerability-indicator-2018/SVI2018_US.csv')
us_cases = pd.read_csv('../input/uncover/UNCOVER/USAFacts/confirmed-covid-19-cases-in-us-by-state-and-county.csv')
us_deaths = pd.read_csv('../input/uncover/UNCOVER/USAFacts/confirmed-covid-19-deaths-in-us-by-state-and-county.csv')



In [ ]:
# Write a function that provides a first look at the new DataFrames. 

dataframe_list = [svi, us_cases, us_deaths]

def first_look (lst):
    """ 
    Input: list of pandas dataframes. 
    Output: Name, Shape, NaNs and describe() of the given dataframes.
    """
    for item in lst:
        def get_df_name(df):
            name =[x for x in globals() if globals()[x] is df][0]
            return name
        print('Information for DataFrame ' + get_df_name(item))
        print(item.shape)
        print((np.sum(item.isna() == True))/item.shape[0])
        print(item.describe())
        print('---')
        

first_look(dataframe_list)


### Preparation of the Social Vulnerability Indicators (SVI) dataframe and of selected indicators

In [ ]:
# We are interested in 14 indicators, all of them are percentage values. 
# They include poverty, minorities, unemployment, etc. 
# Each category has a MOE (margin of error) that is not taken into account in this research.
# The value -999 means no value. I am going to treat them as NaNs and drop them. 

# Select relevant columns of SVI, including state, county and county fips. Rename 'STCNTY' to county_fips for mergers.
svi = svi[['STATE', 'STCNTY','COUNTY', 'EP_POV', 'EP_LIMENG', 'EP_AGE65', 'EP_MINRTY', 'EP_UNEMP', 'EP_NOHSDP', 'EP_AGE17', 'EP_DISABL', 'EP_SNGPNT', 'EP_MUNIT', 'EP_MOBILE', 'EP_CROWD', 'EP_NOVEH', 'EP_GROUPQ', 'RPL_THEMES']]
svi = svi.rename(columns = {'STCNTY': 'county_fips'})

# Drop all -999/NaN columns in the dataframe
svi = svi[svi[svi.columns] != -999]
svi = svi.dropna()

# SVI and confirmed cases of Covid-19 in the US

We will now have a look at the confirmed cases of Covid-19 in the US, on a state and county level. 
1. Preparation of the confirmed cases dataset
2. General overview
    1. Which US states have the most confirmed cases of Covid-19?
    2. Identify correlation between SVI and Covid-19 confirmed cases in all US counties
3. Analysis of the ten US counties with most confirmed cases
    1. Which are the ten US counties with most confirmed cases
    2. Identify correlation between SVI and the number of Covid-19 in the ten most affected US counties
4. Building a Machine Learning model for confirmed cases
    1. Multiple Linear Regression for confirmed cases all US counties
    2. Multiple Linear Regression for the ten US counties with most confirmed cases

## 1. Preparation of Covid-19 confirmed cases dataset in US states and counties

In [ ]:
# Apply datetime
format_str = '%Y-%m-%d'
us_cases['datetime'] = us_cases.date.apply(lambda date: datetime.datetime.strptime(date, format_str))

# Rename column 'confirmed' in 'confirmed_cases' for better understanding:
us_cases = us_cases.rename(columns = {'confirmed': 'confirmed_cases'})

# Drop rows that do not refer to a specific county ('Statewide Unallocated'):
us_cases = us_cases[us_cases.county_name != 'Statewide Unallocated']

us_cases.head()

In [ ]:
# For which counties do we have missing values? 
us_cases[us_cases.confirmed_cases.isna() == True]

In [ ]:
# Weston County appears to have no information for April.
# However there have been no confirmed cases in Weston County: 
print(us_cases[us_cases.county_name == 'Weston County'].confirmed_cases.mean())

## 2. General Overview
### A. Which US states have the most confirmed cases of Covid-19?

In [ ]:
# First group by county and state, then sum up all the counties' confirmed cases to get to the total sum for a state
us_cases_grouping_county = us_cases.groupby(['county_fips', 'state_name']).confirmed_cases.max().reset_index().sort_values('confirmed_cases', ascending = False)
us_cases_grouping_states = us_cases_grouping_county.groupby('state_name').confirmed_cases.sum().reset_index().sort_values('confirmed_cases', ascending = False)

# Plot the map using plotly

fig = go.Figure(data=go.Choropleth(
    locations=us_cases_grouping_states['state_name'], 
    z = us_cases_grouping_states['confirmed_cases'].astype(float), 
    locationmode = 'USA-states', 
    colorscale = 'blues',
    colorbar_title = "Confirmed cases",
))

fig.update_layout(
    title_text = 'Confirmed cases of Covid-19',
    geo_scope='usa', 
)

fig.show()


In [ ]:
# Show confirmed cases in all US counties
us_cases_all_counties = us_cases.groupby(['county_fips', 'county_name', 'state_name']).confirmed_cases.max().reset_index().sort_values('confirmed_cases', ascending = False)
us_cases_all_counties.head()

### B. Identify correlation between SVI and Covid-19 confirmed cases in all US counties

In [ ]:
# Then we group by Fips, County and State and look at the mean value of the selected indicators. 
svi_all_counties = svi.groupby(['county_fips', 
                                 'COUNTY', 
                                 'STATE']).agg({'EP_POV':'mean',
                                              'EP_LIMENG':'mean',
                                               'EP_AGE65':'mean',
                                               'EP_MINRTY':'mean',
                                               'EP_UNEMP':'mean',
                                               'EP_NOHSDP':'mean',
                                               'EP_AGE17':'mean',
                                               'EP_DISABL':'mean',
                                               'EP_SNGPNT':'mean',
                                               'EP_MUNIT':'mean',
                                               'EP_MOBILE':'mean',
                                               'EP_CROWD':'mean',
                                               'EP_NOVEH':'mean',
                                               'EP_GROUPQ':'mean',
                                                'RPL_THEMES':'mean'
                                              }
                                             ).reset_index()

svi_all_counties_cases = svi_all_counties.merge(us_cases_all_counties, on = 'county_fips').sort_values('confirmed_cases', ascending = False)
svi_all_counties_cases = svi_all_counties_cases.drop(['county_name', 'state_name'], axis = 1)
svi_all_counties_cases.head()



In [ ]:
# Plotting a correlation matrix for all US counties
svi_all_counties_cases_corr = svi_all_counties_cases[['confirmed_cases', 'EP_POV', 'EP_LIMENG', 'EP_AGE65', 'EP_MINRTY', 'EP_UNEMP', 'EP_NOHSDP', 'EP_AGE17', 'EP_DISABL', 'EP_SNGPNT', 'EP_MUNIT', 'EP_MOBILE', 'EP_CROWD', 'EP_NOVEH', 'EP_GROUPQ', 'RPL_THEMES']]
corr = svi_all_counties_cases_corr.corr()
table = corr.style.background_gradient(cmap='Blues').format('{0:,.3f}')
table


In [ ]:
# Export Pandas style object to PNG
# html = table.render()

# path_wkthmltoimage = r'.\wkthmltool\bin\wkhtmltoimage.exe'
# config = imgkit.config(wkhtmltoimage=path_wkthmltoimage)

# imgkit.from_string(html, './corr_confirmed_cases_SVI_all_counties.png',config=config)

## 3. Analysis of the ten US counties with most confirmed cases
### B. Which are the ten US counties with most confirmed cases

In [ ]:
# Which are the 10 most affected US counties? 
us_cases_top_10 = us_cases_all_counties.head(10)
print(us_cases_top_10)

# Preparing a list with the 10 most affected counties: 
top_10_cases_fips_list = us_cases_top_10.county_fips.to_list()

In [ ]:
# Select the timeseries statistics of the 10 most affected US counties
us_cases_top_10_datetime = us_cases[us_cases.county_fips.isin(top_10_cases_fips_list) == True]
us_cases_top_10_datetime.head()

In [ ]:
# Plot Timeline of cases

f, ax = plt.subplots(figsize = (16, 10))
ax = sns.lineplot(x = us_cases_top_10_datetime.datetime, y = us_cases_top_10_datetime.confirmed_cases, hue = us_cases_top_10_datetime.county_name, ci = None)

plt.xlabel('Timeline',fontsize=12)
plt.ylabel('Confirmed Cases',fontsize=12)
plt.title('Confirmed cases in US counties', fontsize = 16)
plt.tick_params(labelsize=12)
ax.yaxis.tick_right()
plt.legend(loc=2, fontsize=12)
plt.grid(False)
ax.yaxis.grid()
sns.despine()

plt.show()


### B. Identify correlation between SVI and the number of Covid-19 in the ten most affected US counties

In [ ]:
# Select the rows in the SVI dataframe that contain the ten counties with most confirmed cases. 
svi_top_10_counties_cases = svi[svi.county_fips.isin(top_10_cases_fips_list) == True]


svi_top_10_counties_cases = svi_top_10_counties_cases.groupby(['county_fips', 
                                                                'COUNTY', 
                                                                'STATE']).agg({'EP_POV':'mean',
                                                                              'EP_LIMENG':'mean',
                                                                               'EP_AGE65':'mean',
                                                                               'EP_MINRTY':'mean',
                                                                               'EP_UNEMP':'mean',
                                                                               'EP_NOHSDP':'mean',
                                                                               'EP_AGE17':'mean',
                                                                               'EP_DISABL':'mean',
                                                                               'EP_SNGPNT':'mean',
                                                                               'EP_MUNIT':'mean',
                                                                               'EP_MOBILE':'mean',
                                                                               'EP_CROWD':'mean',
                                                                               'EP_NOVEH':'mean',
                                                                               'EP_GROUPQ':'mean', 
                                                                               'RPL_THEMES': 'mean'
                                                                              }
                                                                             ).reset_index()


# Merge county SVI with confirmed cases of Covid-19

svi_top_10_counties_cases = svi_top_10_counties_cases.merge(us_cases_top_10, on = 'county_fips').sort_values('confirmed_cases', ascending = False)
svi_top_10_counties_cases = svi_top_10_counties_cases.drop(['county_name', 'state_name'], axis = 1)
svi_top_10_counties_cases

In [ ]:
# Plotting a correlation matrix for the ten most affected US counties

svi_top_10_counties_cases_corr = svi_top_10_counties_cases[['confirmed_cases', 'EP_POV', 'EP_LIMENG', 'EP_AGE65', 'EP_MINRTY', 'EP_UNEMP', 'EP_NOHSDP', 'EP_AGE17', 'EP_DISABL', 'EP_SNGPNT', 'EP_MUNIT', 'EP_MOBILE', 'EP_CROWD', 'EP_NOVEH', 'EP_GROUPQ', 'RPL_THEMES']]
corr = svi_top_10_counties_cases_corr.corr()
table = corr.style.background_gradient(cmap='Blues')
table

table = corr.style.background_gradient(cmap='Blues').format('{0:,.3f}')
table


In [ ]:
# Export Pandas style object to PNG
# html = table.render()

#path_wkthmltoimage = r'.\wkthmltool\bin\wkhtmltoimage.exe'
#config = imgkit.config(wkhtmltoimage=path_wkthmltoimage)

#imgkit.from_string(html, './corr_confirmed_cases_SVI_top_10.png',config=config)

In [ ]:
# Visualization of Social Vulnerability Indicators and confirmed Covid-19 cases in the ten most affected counties
# For the visualization, we take RPL_THEMES, which is defined as an overall ranking variable. 
svi_top_10_counties_cases_RPL = svi_top_10_counties_cases.sort_values('confirmed_cases')
sns.set_palette('muted')

f, ax = plt.subplots(figsize = (16, 10))
c = sns.scatterplot(x = svi_top_10_counties_cases_RPL.RPL_THEMES, y = svi_top_10_counties_cases_RPL.confirmed_cases, size = svi_top_10_counties_cases_RPL.RPL_THEMES, sizes = (100, 800), hue = svi_top_10_counties_cases_RPL['COUNTY'])
ax.yaxis.grid()

plt.xlabel('Social Vulnerability Index, overall ranking variable PRL_THEMES', fontsize = 12)
plt.ylabel('Confirmed cases of Covid-19', fontsize = 12)
plt.title('Confirmed cases of Covid-19 and overall SVI ranking in the ten most affected counties', fontsize = 14)
sns.despine()
plt.legend(scatterpoints=1, frameon=True, labelspacing=1.5, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


## 4. Building a Machine Learning model for confirmed cases 
### A. Multiple Linear Regression for confirmed cases all US counties

In [ ]:
# Building the Multiple Linear Regression Model

# Preparing the independent variable (X) and the dependent variables (y)
y = svi_all_counties_cases['confirmed_cases']
X = svi_all_counties_cases.drop(['confirmed_cases', 'county_fips', 'COUNTY', 'STATE', 'RPL_THEMES'], axis = 1)

# Splitting the data in train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state=42)


# Initiate the model
lm = LinearRegression()
model = lm.fit(X_train, y_train)

y_predict= lm.predict(X_test)


# Score
print(lm.score(X_test, y_test))
# Score is very low, which may be due to the fact, that many US counties dont have confirmed cases yet. 

confirmed_cases_all_counties_coef = pd.DataFrame({'Coef' : model.coef_, 'Features' : X.columns}).sort_values('Coef', ascending = False)
confirmed_cases_all_counties_coef

### B. Multiple Linear Regression for the ten US counties with most confirmed cases

In [ ]:
# Building the Multiple Linear Regression Model

# Preparing the independent variable (X) and the dependent variables (y)
y = svi_top_10_counties_cases['confirmed_cases']
X = svi_top_10_counties_cases.drop(['confirmed_cases', 'county_fips', 'EP_POV', 'COUNTY', 'STATE', 'RPL_THEMES'], axis = 1)

# Splitting the data in train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state=42)


# Initiate the model
lm = LinearRegression()
model = lm.fit(X_train, y_train)

y_predict= lm.predict(X_test)


# Score
print(lm.score(X_test, y_test))

confirmed_cases_top_10_counties_coef = pd.DataFrame({'Coef' : model.coef_, 'Features' : X.columns}).sort_values('Coef', ascending = False)
confirmed_cases_top_10_counties_coef

# SVI and Covid-19 deaths in the US

We will now have a look at the deaths of Covid-19 in the US, on a state and county level. 
1. Preparation of the deaths dataset
2. General overview
    1. Which US states have suffered most deaths caused by Covid-19?
    2. Identify correlation between SVI and Covid-19 in all US counties
3. Analysis of the ten US counties with most deaths caused by Covid-19
    1. Which are the ten US counties with most deaths caused by Covid-19? 
    2. Identify correlation between SVI and Covid-19 in the ten most affected US counties
4. Building a Machine Learning model for deaths caused by Covid-19 
    1. Multiple Linear Regression for deaths caused by Covid-19 in all US counties
    2. Multiple Linear Regression for the ten US counties with most deaths caused by Covid-19
    
  

## 1. Preparation of the deaths dataset

In [ ]:
# Apply datetime
format_str = '%Y-%m-%d'
us_deaths['datetime'] = us_deaths.date.apply(lambda date: datetime.datetime.strptime(date, format_str))

# Drop rows that do not refer to a specific county ('Statewide Unallocated'):
us_deaths = us_deaths[us_deaths.county_name != 'Statewide Unallocated']

us_deaths.head()

In [ ]:
# For which counties do we have missing values? 
us_deaths[us_deaths.deaths.isna() == True]

In [ ]:
# Weston County appears to have no information for April. 
# However there have been no confirmed cases in Weston County: 
print(us_deaths[us_deaths.county_name == 'Weston County'].deaths.mean())

## 2. General Overview
### A. Which US states have suffered most deaths caused by Covid-19?

In [ ]:
# First group by county and state, then sum up all the counties' deaths to get to the total sum for a state
us_deaths_grouping_county = us_deaths.groupby(['county_fips', 'state_name']).deaths.max().reset_index().sort_values('deaths', ascending = False)
us_deaths_grouping_states = us_deaths_grouping_county.groupby('state_name').deaths.sum().reset_index().sort_values('deaths', ascending = False)

# Plot the map using plotly

fig = go.Figure(data=go.Choropleth(
    locations=us_deaths_grouping_states['state_name'], 
    z = us_deaths_grouping_states['deaths'].astype(float), 
    locationmode = 'USA-states', 
    colorscale = 'blues',
    colorbar_title = "Deaths",
))

fig.update_layout(
    title_text = 'Deaths caused by Covid-19 in the US states',
    geo_scope='usa', 
)

fig.show()

In [ ]:
# Show deaths in all US counties
us_deaths_all_counties = us_deaths.groupby(['county_fips', 'county_name', 'state_name']).deaths.max().reset_index().sort_values('deaths', ascending = False)
us_deaths_all_counties.head()

## B. Identify correlation between SVI and Covid-19 in all US counties

In [ ]:
svi_all_counties_deaths = svi_all_counties.merge(us_deaths_all_counties, on = 'county_fips').sort_values('deaths', ascending = False)
svi_all_counties_deaths = svi_all_counties_deaths.drop(['county_name', 'state_name'], axis = 1)
svi_all_counties_deaths.head()


In [ ]:
# Plotting a correlation matrix for all US counties
svi_all_counties_deaths_corr = svi_all_counties_deaths[['deaths', 'EP_POV', 'EP_LIMENG', 'EP_AGE65', 'EP_MINRTY', 'EP_UNEMP', 'EP_NOHSDP', 'EP_AGE17', 'EP_DISABL', 'EP_SNGPNT', 'EP_MUNIT', 'EP_MOBILE', 'EP_CROWD', 'EP_NOVEH', 'EP_GROUPQ', 'RPL_THEMES']]
corr = svi_all_counties_deaths_corr.corr()
table = corr.style.background_gradient(cmap='Reds').format('{0:,.3f}')

table

In [ ]:
# Export Pandas style object to PNG
# html = table.render()

# path_wkthmltoimage = r'.\wkthmltool\bin\wkhtmltoimage.exe'
# config = imgkit.config(wkhtmltoimage=path_wkthmltoimage)

# imgkit.from_string(html, './corr_deaths_all_counties.png',config=config)

## 3. Analysis of the ten US counties with most deaths caused by Covid-19
### B. Which are the ten US counties with most deaths caused by Covid-19

In [ ]:
# Which are the 10 most affected US counties? 
us_deaths_top_10 = us_deaths_all_counties.head(10)
print(us_deaths_top_10)


# Preparing a list with the 10 most affected counties: 
top_10_deaths_fips_list = us_deaths_top_10.county_fips.to_list()

In [ ]:
# Select the timeseries statistics of the 10 most affected US counties
us_deaths_top_10_datetime = us_deaths[us_deaths.county_fips.isin(top_10_deaths_fips_list) == True]
us_deaths_top_10_datetime.head()

In [ ]:
# Plot Timeline of deaths

f, ax = plt.subplots(figsize = (16, 10))
ax = sns.lineplot(x = us_deaths_top_10_datetime.datetime, y = us_deaths_top_10_datetime.deaths, hue = us_deaths_top_10_datetime.county_name, ci = None)

plt.xlabel('Timeline',fontsize=12)
plt.ylabel('Deaths',fontsize=12)
plt.title('Deaths caused by Covid-19 in US counties', fontsize = 16)
plt.tick_params(labelsize=12)
ax.yaxis.tick_right()
plt.legend(loc=2, fontsize=12)
plt.grid(False)
ax.yaxis.grid()
sns.despine()

plt.show()

### B. Identify correlation between SVI and the ten US counties with most deaths caused by Covid-19

In [ ]:
# Select the rows in the SVI dataframe that contain the ten counties with most confirmed cases. 
svi_top_10_counties_deaths = svi[svi.county_fips.isin(top_10_deaths_fips_list) == True]

# Drop rows with -999 in the top ten counties:
svi_top_10_counties_deaths = svi_top_10_counties_deaths[svi_top_10_counties_deaths[svi_top_10_counties_deaths.columns] != -999]
svi_top_10_counties_deaths = svi_top_10_counties_deaths.dropna()



svi_top_10_counties_deaths = svi_top_10_counties_deaths.groupby(['county_fips', 
                                                                'COUNTY', 
                                                                'STATE']).agg({'EP_POV':'mean',
                                                                              'EP_LIMENG':'mean',
                                                                               'EP_AGE65':'mean',
                                                                               'EP_MINRTY':'mean',
                                                                               'EP_UNEMP':'mean',
                                                                               'EP_NOHSDP':'mean',
                                                                               'EP_AGE17':'mean',
                                                                               'EP_DISABL':'mean',
                                                                               'EP_SNGPNT':'mean',
                                                                               'EP_MUNIT':'mean',
                                                                               'EP_MOBILE':'mean',
                                                                               'EP_CROWD':'mean',
                                                                               'EP_NOVEH':'mean',
                                                                               'EP_GROUPQ':'mean', 
                                                                               'RPL_THEMES': 'mean'
                                                                              }
                                                                             ).reset_index()


# Merge county SVI with deaths caused by Covid-19
svi_top_10_counties_deaths = svi_top_10_counties_deaths.merge(us_deaths_top_10, on = 'county_fips').sort_values('deaths', ascending = False)
svi_top_10_counties_deaths = svi_top_10_counties_deaths.drop(['county_name', 'state_name'], axis = 1)
svi_top_10_counties_deaths

In [ ]:
# Plotting a correlation matrix for the ten most affected US counties

svi_top_10_counties_deaths_corr = svi_top_10_counties_deaths[['deaths', 'EP_POV', 'EP_LIMENG', 'EP_AGE65', 'EP_MINRTY', 'EP_UNEMP', 'EP_NOHSDP', 'EP_AGE17', 'EP_DISABL', 'EP_SNGPNT', 'EP_MUNIT', 'EP_MOBILE', 'EP_CROWD', 'EP_NOVEH', 'EP_GROUPQ', 'RPL_THEMES']]
corr = svi_top_10_counties_deaths_corr.corr()
table = corr.style.background_gradient(cmap='Reds').format('{0:,.3f}')

table


In [ ]:
# Export Pandas style object to PNG
# html = table.render()

# path_wkthmltoimage = r'.\wkthmltool\bin\wkhtmltoimage.exe'
# config = imgkit.config(wkhtmltoimage=path_wkthmltoimage)

# imgkit.from_string(html, './corr_deaths_top_10_counties.png',config=config)

In [ ]:
# Visualization of Social Vulnerability Indicators and deaths caused by Covid-19 in the ten most affected counties
# For the visualization, we take RPL_THEMES, which is defined as an overall ranking variable. 
svi_top_10_counties_deaths_RPL = svi_top_10_counties_deaths.sort_values('deaths')
sns.set_palette('muted')

f, ax = plt.subplots(figsize = (16, 10))
c = sns.scatterplot(x = svi_top_10_counties_deaths_RPL.RPL_THEMES, y = svi_top_10_counties_deaths_RPL.deaths, size = svi_top_10_counties_deaths_RPL.RPL_THEMES, sizes = (100, 800), hue = svi_top_10_counties_deaths_RPL['COUNTY'])
ax.yaxis.grid()

plt.xlabel('Social Vulnerability Index, overall ranking variable PRL_THEMES', fontsize = 12)
plt.ylabel('Deaths caused by Covid-19 as of April 8th, 2020', fontsize = 12)
plt.title('Deaths caused by Covid-19 and overall SVI ranking in the ten most affected counties', fontsize = 14)
sns.despine()
plt.legend(scatterpoints=1, frameon=True, labelspacing=1.5, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


## 4. Building a Machine Learning model for deaths caused by Covid-19 
### A. Multiple Linear Regression for deaths caused by Covid-19 in all US counties

In [ ]:
# Building the Multiple Linear Regression Model

# Preparing the independent variable (X) and the dependent variables (y)
y = svi_all_counties_deaths['deaths']
X = svi_all_counties_deaths.drop(['deaths', 'county_fips', 'COUNTY', 'STATE', 'RPL_THEMES'], axis = 1)

# Splitting the data in train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state=42)


# Initiate the model
lm = LinearRegression()
model = lm.fit(X_train, y_train)

y_predict= lm.predict(X_test)


# Score
print(lm.score(X_test, y_test))
# Score is very low, which may be due to the fact, that many US counties dont have confirmed cases yet. 

deaths_all_counties_coef = pd.DataFrame({'Coef' : model.coef_, 'Features' : X.columns}).sort_values('Coef', ascending = False)
deaths_all_counties_coef

### B. Multiple Linear Regression for the ten US counties with most deaths caused by Covid-19

In [ ]:
# Building the Multiple Linear Regression Model

# Preparing the independent variable (X) and the dependent variables (y)
y = svi_top_10_counties_deaths['deaths']
X = svi_top_10_counties_deaths.drop(['deaths', 'county_fips', 'COUNTY', 'STATE', 'RPL_THEMES'], axis = 1)

# Splitting the data in train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state=42)


# Initiate the model
lm = LinearRegression()
model = lm.fit(X_train, y_train)

y_predict= lm.predict(X_test)


# Score
print(lm.score(X_test, y_test))

deaths_top_10_counties_coef = pd.DataFrame({'Coef' : model.coef_, 'Features' : X.columns}).sort_values('Coef', ascending = False)
deaths_top_10_counties_coef